<a href="https://colab.research.google.com/github/raja-dettex/dd-colorizer/blob/main/Another_copy_of_colorization_pipeline_onnxruntime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/piddnad/DDColor.git

# **after first time you are done processed a set of images run only the second cell and last cell. **
here is the flow
for first time : run first cell -> run second cell -> upload images to 'images' folder(the 'images' folder is located in 'content' folder)  -> run rest of the cells one by one. (doing this will set your weights model and path and onnxruntime and ort_session to prepare the tensor)





second time onwards: run second cell -> upload images to 'images' folder -> run the last cell
## don't forget to download images from 'finalized' folder before running the next processing task

In [ ]:
import os
weights_dir_path = '/content/DDColor/weights/'
input_path = '/content/images'
final_path = '/content/finalized/'
os.makedirs(input_path, exist_ok=True)
os.makedirs(weights_dir_path, exist_ok=True)
os.makedirs(final_path, exist_ok=True)
# cleanup after every batch task is complete
for file in os.listdir(input_path):
  os.remove(os.path.join(input_path, file))
for file in os.listdir(final_path):
  os.remove(os.path.join(final_path, file))

In [ ]:
!pip install onnx onnxruntime onnxsim

In [ ]:
import numpy as np
import os
import onnxruntime as ort
import tqdm
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
model_path = '/content/DDColor/weights/ddcolor-tiny-op12.onnx'    # python export.py --model_path pretrain/ddcolor_paper_tiny.pth --export_path weights/ddcolor-tiny-op12.onnx

#Load some example image


In [ ]:
!wget -nc https://huggingface.co/piddnad/DDColor-models/resolve/main/ddcolor_paper_tiny.pth -P /content/DDColor/pretrain/

File ‘/content/DDColor/pretrain/ddcolor_paper_tiny.pth’ already there; not retrieving.



In [ ]:
!python /content/DDColor/export.py --model_path /content/DDColor/pretrain/ddcolor_paper_tiny.pth --export_path /content/DDColor/weights/ddcolor-tiny-op12.onnx

/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
ONNX file successfully created at /content/DDColor/weights/ddcolor-tiny-op12.onnx
ONNX file at /content/DDColor/weights/ddcolor-tiny-op12.onnx verifed shapes and simplified


In [ ]:

tmp_ort_session = ort.InferenceSession(model_path, providers=['CPUExecutionProvider'])

# print the input,output names and shapes
for i in range(len(tmp_ort_session.get_inputs())):
    print(f"Input name: {tmp_ort_session.get_inputs()[i].name}, shape: {tmp_ort_session.get_inputs()[i].shape}")
for i in range(len(tmp_ort_session.get_outputs())):
    print(f"Output name: {tmp_ort_session.get_outputs()[i].name}, shape: {tmp_ort_session.get_outputs()[i].shape}")


providers = [
    # The TensorrtExecutionProvider is the fastest.
    ('TensorrtExecutionProvider', {
        'device_id': 0,
        'trt_max_workspace_size': 4 * 1024 * 1024 * 1024,
        'trt_fp16_enable': True,
        'trt_engine_cache_enable': True,
        'trt_engine_cache_path': './trt_engine_cache',
        'trt_engine_cache_prefix': 'model',
        'trt_dump_subgraphs': False,
        'trt_timing_cache_enable': True,
        'trt_timing_cache_path': './trt_engine_cache',
        #'trt_builder_optimization_level': 3,
    }),

    # The CUDAExecutionProvider is slower than PyTorch,
    # possibly due to performance issues with large matrix multiplication "cossim = torch.bmm(feats1, feats2.permute(0,2,1))"
    # Reducing the top_k value when exporting to ONNX can decrease the matrix size.
    ('CUDAExecutionProvider', {
        'device_id': 0,
        'gpu_mem_limit': 4 * 1024 * 1024 * 1024,
    }),
    ('CPUExecutionProvider',{
    })
]

ort_session = ort.InferenceSession(model_path, providers=providers)

Input name: input, shape: [1, 3, 512, 512]
Output name: output, shape: [1, 2, 512, 512]


/usr/local/lib/python3.11/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:121: UserWarning: Specified provider 'TensorrtExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:121: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


## Prepare input tensor

In [ ]:
def process_single_image(ort_session, img_path, outfile_name):
  input_size = 512
  batch_size = 1
  img = cv2.imread(img_path, cv2.IMREAD_COLOR)
  height, width = img.shape[:2]
  # print(self.width, self.height)
  # if self.width * self.height < 100000:
  #     self.input_size = 256

  img = (img / 255.0).astype(np.float32)
  orig_l = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)[:, :, :1]  # (h, w, 1)

  # resize rgb image -> lab -> get grey -> rgb
  img_resize = cv2.resize(img, (input_size, input_size))
  img_l = cv2.cvtColor(img_resize, cv2.COLOR_BGR2Lab)[:, :, :1]
  img_gray_lab = np.concatenate((img_l, np.zeros_like(img_l), np.zeros_like(img_l)), axis=-1)
  img_gray_rgb = cv2.cvtColor(img_gray_lab, cv2.COLOR_LAB2RGB)

  img_gray_rgb = img_gray_rgb.transpose((2, 0, 1)).astype(np.float32)
  img_gray_rgb = np.expand_dims(img_gray_rgb, axis=0)

  # Psuedo-batch the input images
  img_gray_rgb = np.concatenate([img_gray_rgb for _ in range(batch_size)], axis=0)

  inputs = {
      ort_session.get_inputs()[0].name: img_gray_rgb,
  }

  # output_ab = self.model(tensor_gray_rgb).cpu()  # (1, 2, self.height, self.width)
  output_ab = torch.from_numpy(ort_session.run(None, inputs)[0])

  output_ab_resize = F.interpolate(output_ab, size=(height, width))[0].float().numpy().transpose(1, 2, 0)
  output_lab = np.concatenate((orig_l, output_ab_resize), axis=-1)
  output_rgb = cv2.cvtColor(output_lab, cv2.COLOR_LAB2RGB)

  output_img = (output_rgb * 255.0).round().astype(np.uint8)
  outfilepath = f'/content/finalized/{outfile_name}'
  cv2.imwrite(outfilepath, output_img)

## Process output

In [ ]:
input_dir = '/content/images'
image_files = os.listdir(input_dir)
from concurrent.futures import ThreadPoolExecutor
output_dir = '/content/finalized'
process_count = 0
with ThreadPoolExecutor(max_workers=4) as executor:
  futures = [executor.submit(process_single_image, ort_session, os.path.join(input_dir, image_file), image_file) for image_file in image_files]
  for future in futures:
    result = future.result
    if result:
      process_count += 1
print(f'finished processing {process_count} images')


finished processing 4 images
